In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Pará - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [5]:
data = pd.read_csv('2003_01_model_input_PA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data 

,Ano,Pará - Valor Cimento Portland,Pará - Produção de Cimento (t),Pará - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Pará - Consumo de Cimento (t)
0,2003,3.664692,618.075773,8.348543,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,...,NaN,NaN,NaN,NaN,0.669564,7.776091e+07,4.250430e+06,9.220374,6.851269e+07,1052.310000
1,2004,3.365934,678.161979,8.276683,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,...,27.496357,23.372183,79.344081,1.011025,0.671958,7.909171e+07,4.310972e+06,9.281804,6.912419e+07,1193.163000
2,2005,3.175303,747.048967,8.254638,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,...,27.230294,23.379773,80.515682,1.001162,0.674195,8.028032e+07,4.348769e+06,9.324669,6.956367e+07,1264.982000
3,2006,3.383776,874.128576,8.168506,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,...,27.122091,23.230854,80.386131,1.123033,0.676282,8.143162e+07,4.390026e+06,9.371675,7.001147e+07,1181.617000
4,2007,3.978395,1030.601051,8.144568,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,...,27.084070,23.115122,80.063166,1.208326,0.678225,8.248830e+07,4.436626e+06,9.412493,7.039051e+07,1272.803000
5,2008,4.409578,1172.693755,8.116333,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,...,27.010379,23.052703,80.170200,1.147919,0.680030,8.343245e+07,4.475828e+06,9.446969,7.069042e+07,1559.491000
6,2009,4.558849,1310.052663,8.148790,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,...,26.957054,23.034986,80.332906,1.075145,0.681704,8.418073e+07,4.507563e+06,9.465568,7.086672e+07,1355.544000
7,2010,4.738253,1443.581473,8.112071,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,...,26.838317,22.916120,80.307326,1.001921,0.683252,8.675706e+07,4.555784e+06,9.542119,7.180858e+07,1640.682000
8,2011,4.839096,1536.079376,8.099477,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,...,26.780148,22.842507,80.217883,0.944877,0.684681,8.890935e+07,4.574157e+06,9.572272,7.235179e+07,1856.176000
9,2012,4.865765,1569.763418,9.368278,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,...,26.730543,22.851410,80.485786,0.886023,0.685697,9.072117e+07,4.559887e+06,9.574043,7.265948e+07,2042.910000


In [6]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Pará - Valor Cimento Portland,Pará - Produção de Cimento (t),Pará - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
0,3.664692,618.075773,8.348543,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,...,NaN,NaN,NaN,NaN,NaN,0.669564,7.776091e+07,4.250430e+06,9.220374,6.851269e+07
1,3.365934,678.161979,8.276683,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,...,14053.690695,27.496357,23.372183,79.344081,1.011025,0.671958,7.909171e+07,4.310972e+06,9.281804,6.912419e+07
2,3.175303,747.048967,8.254638,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,...,13888.682594,27.230294,23.379773,80.515682,1.001162,0.674195,8.028032e+07,4.348769e+06,9.324669,6.956367e+07
3,3.383776,874.128576,8.168506,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,...,12607.324878,27.122091,23.230854,80.386131,1.123033,0.676282,8.143162e+07,4.390026e+06,9.371675,7.001147e+07
4,3.978395,1030.601051,8.144568,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,...,8627.465979,27.084070,23.115122,80.063166,1.208326,0.678225,8.248830e+07,4.436626e+06,9.412493,7.039051e+07
5,4.409578,1172.693755,8.116333,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,...,4848.884846,27.010379,23.052703,80.170200,1.147919,0.680030,8.343245e+07,4.475828e+06,9.446969,7.069042e+07
6,4.558849,1310.052663,8.148790,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,...,3110.216649,26.957054,23.034986,80.332906,1.075145,0.681704,8.418073e+07,4.507563e+06,9.465568,7.086672e+07
7,4.738253,1443.581473,8.112071,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,...,2155.562456,26.838317,22.916120,80.307326,1.001921,0.683252,8.675706e+07,4.555784e+06,9.542119,7.180858e+07
8,4.839096,1536.079376,8.099477,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,...,1742.787440,26.780148,22.842507,80.217883,0.944877,0.684681,8.890935e+07,4.574157e+06,9.572272,7.235179e+07
9,4.865765,1569.763418,9.368278,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,...,1488.532092,26.730543,22.851410,80.485786,0.886023,0.685697,9.072117e+07,4.559887e+06,9.574043,7.265948e+07


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     1193.163000
1     1264.982000
2     1181.617000
3     1272.803000
4     1559.491000
5     1355.544000
6     1640.682000
7     1856.176000
8     2042.910000
9     2318.608000
10    2041.730714
11    1863.549771
12    1705.680514
13    1564.940000
14    1375.281000
15    1290.928000
16    1444.547000
17    1529.052000
18    1592.624000
19            NaN
Name: Pará - Consumo de Cimento (t), dtype: float64

In [17]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input = train_input.fillna(0.0)
train_input

,Pará - Valor Cimento Portland,Pará - Produção de Cimento (t),Pará - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
0,-1.138757,-1.758406,-0.752091,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,...,0.000000,0.000000,0.000000,0.000000,0.000000,-1.803855,-1.590257,-1.663515,-2.113181,-1.842041
1,-1.451101,-1.560820,-0.822573,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,...,2.018485,2.758165,2.075016,-2.630337,0.572607,-1.508221,-1.391233,-1.059846,-1.650898,-1.502641
2,-1.650401,-1.334294,-0.844196,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,...,1.983973,1.437238,2.119130,0.538899,0.512113,-1.231911,-1.213474,-0.682977,-1.328330,-1.258722
3,-1.432448,-0.916408,-0.928677,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,...,1.715969,0.900038,1.253575,0.188457,1.259573,-0.974158,-1.041295,-0.271603,-0.974596,-1.010184
4,-0.810788,-0.401868,-0.952157,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,...,0.883558,0.711277,0.580910,-0.685178,1.782691,-0.734193,-0.883266,0.193050,-0.667428,-0.799809
5,-0.359997,0.065386,-0.979851,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,...,0.093244,0.345423,0.218113,-0.395645,1.412205,-0.511248,-0.742067,0.583940,-0.407990,-0.633351
6,-0.203938,0.517073,-0.948016,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,...,-0.270409,0.080678,0.115137,0.044481,0.965867,-0.304553,-0.630159,0.900371,-0.268023,-0.535503
7,-0.016376,0.956166,-0.984032,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,...,-0.470080,-0.508816,-0.575741,-0.024714,0.516773,-0.113340,-0.244865,1.381186,0.308043,-0.012746
8,0.089054,1.260334,-0.996384,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,...,-0.556415,-0.797612,-1.003602,-0.266662,0.166905,0.063159,0.077014,1.564381,0.534951,0.288749
9,0.116936,1.371099,0.248107,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,...,-0.609594,-1.043884,-0.951857,0.458028,-0.194058,0.188671,0.347976,1.422092,0.548279,0.459520


In [18]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     1193.163000
1     1264.982000
2     1181.617000
3     1272.803000
4     1559.491000
5     1355.544000
6     1640.682000
7     1856.176000
8     2042.910000
9     2318.608000
10    2041.730714
11    1863.549771
12    1705.680514
13    1564.940000
14    1375.281000
15    1290.928000
16    1444.547000
Name: Pará - Consumo de Cimento (t), dtype: float64

In [19]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Pará - Valor Cimento Portland,Pará - Produção de Cimento (t),Pará - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
17,2.159157,-0.855382,0.928663,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,...,-0.678756,-0.559137,1.493996,3.182386,-1.221366,1.09534,1.061519,-0.73643,1.153715,1.114632


In [20]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    1529.052
Name: Pará - Consumo de Cimento (t), dtype: float64

In [21]:
train_input.iloc[-(len(train_input)//5):]

,Pará - Valor Cimento Portland,Pará - Produção de Cimento (t),Pará - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
14,1.116838,-0.080187,1.168677,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,...,-0.695771,-0.640422,-0.729943,0.241862,-1.187908,1.388567,1.258699,-1.036694,1.073783,1.189219
15,1.227677,-0.577707,1.074766,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,...,-0.713346,-0.872493,-0.435995,1.326206,-1.199990,1.404430,1.284737,-0.912968,1.157312,1.234960
16,1.694083,-0.841081,1.042191,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,...,-0.716923,-0.676399,0.163900,2.269734,-1.226909,1.275773,1.262555,-0.833260,1.268272,1.275259


In [22]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [23]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [24]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1779301585, 2859819855, 1998156311, 4150174205, 2780825035, 3417816891, 2775722920, 718624846, 561620832, 1321339145, 2209203284, 3289778685, 2841059350, 2867752591, 4255661034, 166139991, 228782200, 3364336906, 2187196860, 1946002750, 3057107841, 3380351653, 3652394344, 790611008, 333993799, 1161622245, 1580922423, 2871824976, 2225502186, 2322727117, 1771449459, 722633515, 367937219, 2497930115, 1646016589, 4248910383, 1083335498, 3211281477, 3548883905, 3790077382, 4210917373, 1212303285, 2623137352, 3985672288, 3309483628, 2457041050, 1333280844, 2080813205, 4160579172, 1379853419, 2182596227, 2653246100, 1782271366, 3362265928, 1017455372, 2848926092, 1723783064, 1012857238, 2498659680, 3630589238, 357973280, 3939131562, 1164550607, 479741079, 140672442, 786889763, 733746122, 1388944102, 3619054533, 3348894014, 2060394621, 1587156750, 1723887258, 2710741520, 2854476249, 116959212, 2987333020, 3220466549, 3139132750, 1156933070, 1607099536, 134292321, 3844115536, 1497592674, 393456

val_loss: 10703.4677734375


Step: 87 ___________________________________________
val_loss: 2889.990234375


Step: 88 ___________________________________________
val_loss: 4755.73974609375


Step: 89 ___________________________________________
val_loss: 5482.06787109375


Step: 90 ___________________________________________
val_loss: 88980.71875


Step: 91 ___________________________________________
val_loss: 23810.798828125


Step: 92 ___________________________________________
val_loss: 3946.744140625


Step: 93 ___________________________________________
val_loss: 5675.81591796875


Step: 94 ___________________________________________
val_loss: 27390.6015625


Step: 95 ___________________________________________
val_loss: 15072.3369140625


Step: 96 ___________________________________________
val_loss: 105324.1796875


Step: 97 ___________________________________________
val_loss: 3938.832763671875


Step: 98 ___________________________________________
val_loss: 4462.22802734375


S

In [25]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 475ms/step - loss: 2625853.0000 - val_loss: 1791722.1250
Epoch 2/10000
1/1 [==============================] - 0s 15ms/step - loss: 2546113.7500 - val_loss: 4748815.5000
Epoch 3/10000
1/1 [==============================] - 0s 399ms/step - loss: 3792710.0000 - val_loss: 1134446.7500
Epoch 4/10000
1/1 [==============================] - 0s 16ms/step - loss: 2657508.0000 - val_loss: 1864252.6250
Epoch 5/10000
1/1 [==============================] - 0s 16ms/step - loss: 2594787.5000 - val_loss: 2463466.2500
Epoch 6/10000
1/1 [==============================] - 0s 16ms/step - loss: 2249163.2500 - val_loss: 2427223.7500
Epoch 7/10000
1/1 [==============================] - 0s 16ms/step - loss: 2238749.7500 - val_loss: 2258047.7500
Epoch 8/10000
1/1 [==============================] - 0s 15ms/step - loss: 2115269.2500 - val_loss: 2104126.2500
Epoch 9/10000
1/1 [==============================] - 0s 15ms/step - loss: 2145606.5000 - val_loss: 207

1/1 [==============================] - 0s 16ms/step - loss: 536407.3125 - val_loss: 383807.7500
Epoch 75/10000
1/1 [==============================] - 0s 16ms/step - loss: 509778.1250 - val_loss: 352185.5938
Epoch 76/10000
1/1 [==============================] - 0s 16ms/step - loss: 506777.0000 - val_loss: 347913.7188
Epoch 77/10000
1/1 [==============================] - 0s 15ms/step - loss: 516563.1250 - val_loss: 228476.7969
Epoch 78/10000
1/1 [==============================] - 0s 15ms/step - loss: 467611.9062 - val_loss: 260676.6719
Epoch 79/10000
1/1 [==============================] - 0s 15ms/step - loss: 465414.2500 - val_loss: 244136.8750
Epoch 80/10000
1/1 [==============================] - 0s 15ms/step - loss: 496525.9375 - val_loss: 343955.0938
Epoch 81/10000
1/1 [==============================] - 0s 15ms/step - loss: 442314.0625 - val_loss: 309998.5625
Epoch 82/10000
1/1 [==============================] - 0s 15ms/step - loss: 452612.8750 - val_loss: 239662.9531
Epoch 83/10000
1

Epoch 148/10000
1/1 [==============================] - 0s 17ms/step - loss: 194206.3750 - val_loss: 46473.7500
Epoch 149/10000
1/1 [==============================] - 0s 16ms/step - loss: 197521.6250 - val_loss: 32022.2441
Epoch 150/10000
1/1 [==============================] - 0s 15ms/step - loss: 182083.9219 - val_loss: 22727.1348
Epoch 151/10000
1/1 [==============================] - 0s 15ms/step - loss: 174013.3281 - val_loss: 23591.2090
Epoch 152/10000
1/1 [==============================] - 0s 15ms/step - loss: 175092.2500 - val_loss: 48397.0195
Epoch 153/10000
1/1 [==============================] - 0s 15ms/step - loss: 144143.7344 - val_loss: 20341.4473
Epoch 154/10000
1/1 [==============================] - 0s 15ms/step - loss: 159398.3906 - val_loss: 35382.5312
Epoch 155/10000
1/1 [==============================] - 0s 15ms/step - loss: 169658.3594 - val_loss: 19234.8770
Epoch 156/10000
1/1 [==============================] - 0s 15ms/step - loss: 164101.6406 - val_loss: 26981.0059
E

1/1 [==============================] - 0s 15ms/step - loss: 89844.6328 - val_loss: 5763.1567
Epoch 223/10000
1/1 [==============================] - 0s 16ms/step - loss: 79916.7969 - val_loss: 5029.0464
Epoch 224/10000
1/1 [==============================] - 0s 16ms/step - loss: 77998.9766 - val_loss: 5124.6450
Epoch 225/10000
1/1 [==============================] - 0s 15ms/step - loss: 96371.4141 - val_loss: 6685.6597
Epoch 226/10000
1/1 [==============================] - 0s 15ms/step - loss: 83800.9688 - val_loss: 4781.7544
Epoch 227/10000
1/1 [==============================] - 0s 15ms/step - loss: 85363.8438 - val_loss: 6182.6392
Epoch 228/10000
1/1 [==============================] - 0s 15ms/step - loss: 74609.6172 - val_loss: 5074.8628
Epoch 229/10000
1/1 [==============================] - 0s 15ms/step - loss: 83750.9453 - val_loss: 5893.4116
Epoch 230/10000
1/1 [==============================] - 0s 15ms/step - loss: 78687.8047 - val_loss: 5197.0864
Epoch 231/10000
1/1 [==============

1/1 [==============================] - 0s 16ms/step - loss: 42013.7812 - val_loss: 7344.9180
Epoch 298/10000
1/1 [==============================] - 0s 17ms/step - loss: 43742.3086 - val_loss: 4180.7319
Epoch 299/10000
1/1 [==============================] - 0s 17ms/step - loss: 42589.2695 - val_loss: 4060.7461
Epoch 300/10000
1/1 [==============================] - 0s 16ms/step - loss: 42970.5195 - val_loss: 4003.2292
Epoch 301/10000
1/1 [==============================] - 0s 15ms/step - loss: 39332.8672 - val_loss: 4020.2109
Epoch 302/10000
1/1 [==============================] - 0s 15ms/step - loss: 35774.9023 - val_loss: 4434.0938
Epoch 303/10000
1/1 [==============================] - 0s 15ms/step - loss: 44643.4961 - val_loss: 3996.5986
Epoch 304/10000
1/1 [==============================] - 0s 15ms/step - loss: 39821.1406 - val_loss: 3977.4714
Epoch 305/10000
1/1 [==============================] - 0s 16ms/step - loss: 41010.5469 - val_loss: 3959.9016
Epoch 306/10000
1/1 [==============

...

Epoch 970/10000
1/1 [==============================] - 0s 16ms/step - loss: 17410.0391 - val_loss: 20149.2695
Epoch 971/10000
1/1 [==============================] - 0s 15ms/step - loss: 26577.1914 - val_loss: 36699.2930
Epoch 972/10000
1/1 [==============================] - 0s 15ms/step - loss: 26665.3047 - val_loss: 33498.5273
Epoch 973/10000
1/1 [==============================] - 0s 15ms/step - loss: 23346.6055 - val_loss: 47466.7812
Epoch 974/10000
1/1 [==============================] - 0s 15ms/step - loss: 20889.4512 - val_loss: 39760.4102
Epoch 975/10000
1/1 [==============================] - 0s 15ms/step - loss: 23065.6426 - val_loss: 37798.7891
Epoch 976/10000
1/1 [==============================] - 0s 15ms/step - loss: 18161.8711 - val_loss: 37155.1289
Epoch 977/10000
1/1 [==============================] - 0s 15ms/step - loss: 18442.8125 - val_loss: 36540.4961
Epoch 978/10000
1/1 [==============================] - 0s 15ms/step - loss: 16005.0000 - val_loss: 30554.7949
Epoch 979/

1/1 [==============================] - 0s 15ms/step - loss: 15828.1787 - val_loss: 18620.5293
Epoch 1045/10000
1/1 [==============================] - 0s 15ms/step - loss: 14317.8291 - val_loss: 17675.2969
Epoch 1046/10000
1/1 [==============================] - 0s 15ms/step - loss: 21125.1406 - val_loss: 19030.2871
Epoch 1047/10000
1/1 [==============================] - 0s 15ms/step - loss: 11823.9502 - val_loss: 13944.5322
Epoch 1048/10000
1/1 [==============================] - 0s 15ms/step - loss: 13665.6699 - val_loss: 14266.4570
Epoch 1049/10000
1/1 [==============================] - 0s 15ms/step - loss: 13056.9922 - val_loss: 15514.2412
Epoch 1050/10000
1/1 [==============================] - 0s 15ms/step - loss: 20604.8691 - val_loss: 12984.9844
Epoch 1051/10000
1/1 [==============================] - 0s 15ms/step - loss: 13628.0781 - val_loss: 14613.1719
Epoch 1052/10000
1/1 [==============================] - 0s 15ms/step - loss: 10254.7559 - val_loss: 15868.9678
Epoch 1053/10000
1

Epoch 1118/10000
1/1 [==============================] - 0s 16ms/step - loss: 15568.9668 - val_loss: 36527.7891
Epoch 1119/10000
1/1 [==============================] - 0s 15ms/step - loss: 17228.1973 - val_loss: 35303.1797
Epoch 1120/10000
1/1 [==============================] - 0s 15ms/step - loss: 16952.5859 - val_loss: 33483.0273
Epoch 1121/10000
1/1 [==============================] - 0s 15ms/step - loss: 9386.7500 - val_loss: 40600.4180
Epoch 1122/10000
1/1 [==============================] - 0s 15ms/step - loss: 21666.5156 - val_loss: 28858.5645
Epoch 1123/10000
1/1 [==============================] - 0s 15ms/step - loss: 16643.7773 - val_loss: 24551.8281
Epoch 1124/10000
1/1 [==============================] - 0s 15ms/step - loss: 15012.9297 - val_loss: 24456.8125
Epoch 1125/10000
1/1 [==============================] - 0s 15ms/step - loss: 15403.2295 - val_loss: 26448.6504
Epoch 1126/10000
1/1 [==============================] - 0s 15ms/step - loss: 18035.0977 - val_loss: 20289.5449
Ep

Epoch 1192/10000
1/1 [==============================] - 0s 16ms/step - loss: 20049.2500 - val_loss: 44328.9414
Epoch 1193/10000
1/1 [==============================] - 0s 15ms/step - loss: 13316.6533 - val_loss: 32634.6855
Epoch 1194/10000
1/1 [==============================] - 0s 15ms/step - loss: 14823.1230 - val_loss: 39678.8516
Epoch 1195/10000
1/1 [==============================] - 0s 15ms/step - loss: 22582.6211 - val_loss: 38128.8438
Epoch 1196/10000
1/1 [==============================] - 0s 17ms/step - loss: 17422.5742 - val_loss: 31703.6875
Epoch 1197/10000
1/1 [==============================] - 0s 20ms/step - loss: 17390.4258 - val_loss: 41910.2148
Epoch 1198/10000
1/1 [==============================] - 0s 19ms/step - loss: 16068.2314 - val_loss: 44773.9336
Epoch 1199/10000
1/1 [==============================] - 0s 18ms/step - loss: 10597.1240 - val_loss: 38027.1211
Epoch 1200/10000
1/1 [==============================] - 0s 17ms/step - loss: 13135.6807 - val_loss: 50528.9531
E

Epoch 1266/10000
1/1 [==============================] - 0s 16ms/step - loss: 20767.6270 - val_loss: 31732.0527
Epoch 1267/10000
1/1 [==============================] - 0s 16ms/step - loss: 15535.7939 - val_loss: 19170.1113
Epoch 1268/10000
1/1 [==============================] - 0s 16ms/step - loss: 18922.8047 - val_loss: 17568.2090
Epoch 1269/10000
1/1 [==============================] - 0s 24ms/step - loss: 21283.9707 - val_loss: 78614.1875
Epoch 1269: early stopping


In [26]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 57ms/step
[1529.052](test_target) - [[1399.0402]](prediction) = 130.0118388671874


In [29]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            train_input = train_input.fillna(0.0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [30]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Pará - Valor Cimento Portland,Pará - Produção de Cimento (t),Pará - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
0,-0.434695,-1.283831,1.884449,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,...,0.000000,0.000000,0.000000,0.000000,0.000000,-1.635956,-1.529973,-1.640359,-1.656369,-1.638970
1,-0.965518,-1.073189,0.995176,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,...,1.170077,1.984420,1.314408,-2.269682,-0.928403,-1.102232,-1.041349,-1.016036,-1.031493,-1.015819
2,-1.304224,-0.831695,0.722359,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,...,1.135497,0.633562,1.360853,0.989707,-1.058858,-0.603396,-0.604930,-0.626272,-0.595472,-0.567975
3,-0.933818,-0.386198,-0.343539,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,...,0.866973,0.084189,0.449551,0.629297,0.553048,-0.138061,-0.182213,-0.200823,-0.117323,-0.111649
4,0.122681,0.162340,-0.639777,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,...,0.032944,-0.108850,-0.258666,-0.269190,1.681158,0.295160,0.205764,0.279729,0.297881,0.274608
5,0.888792,0.660468,-0.989192,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,...,-0.758904,-0.482993,-0.640639,0.028580,0.882200,0.697655,0.552423,0.683993,0.648569,0.580233
6,1.154012,1.142000,-0.587533,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,...,-1.123263,-0.753738,-0.749057,0.481226,-0.080334,1.070812,0.827169,1.011251,0.837764,0.759885
7,1.472770,1.610106,-1.041943,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,...,-1.323323,-1.356590,-1.476450,0.410062,-1.048810,1.416018,1.773109,1.508517,1.616443,1.719687


train_target:


,Pará - Consumo de Cimento (t)
0,1193.163
1,1264.982
2,1181.617
3,1272.803
4,1559.491
5,1355.544
6,1640.682
7,1856.176


test_input:


...

,Pará - Valor Cimento Portland,Pará - Produção de Cimento (t),Pará - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
17,2.159157,-0.855382,0.928663,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,...,-0.678756,-0.559137,1.493996,3.182386,-1.221366,1.09534,1.061519,-0.73643,1.153715,1.114632


test_target:


,Pará - Consumo de Cimento (t)
17,1529.052


1/1 [==============================] - 0s 34ms/step
Error: 130.0118388671874




[1612.0570068359375,
 1694.1182861328125,
 2139.0693359375,
 2126.151611328125,
 1947.9757080078125,
 1905.087646484375,
 1635.109130859375,
 1572.720947265625,
 1497.767578125,
 1399.0401611328125]

In [31]:
display(targets)
display(predictions)

[2042.91,
 2318.608,
 2041.730714285714,
 1863.549771428571,
 1705.6805142857138,
 1564.94,
 1375.281,
 1290.928,
 1444.547,
 1529.052]

[1612.0570068359375,
 1694.1182861328125,
 2139.0693359375,
 2126.151611328125,
 1947.9757080078125,
 1905.087646484375,
 1635.109130859375,
 1572.720947265625,
 1497.767578125,
 1399.0401611328125]

In [32]:
mae = mean_absolute_error(predictions, targets)
mae

272.2579503906251

In [33]:
porcentage = mae/np.mean(targets)
porcentage

0.1584993610380914